# Стреляем себе в ногу
 
* Давайте разберём на PyTorch несколько архитектур. Попытайтесь понять что именно с ними не так. 
* Упражнения позаимствованы [из ШАДОвского семинара.](https://github.com/yandexdataschool/Practical_DL/blob/spring20/seminar03-conv_nets/how_to_shoot_yourself_in_the_foot_with_cnn.ipynb)

#### a) Задача классификации картинок

In [5]:
import torch
import torch.nn as nn

In [6]:
# assuming input shape [batch, 3, 64, 64]
cnn = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=2048, kernel_size=(3,3)),  # много каналов в начале, мало в конце
    # нет нелинейностей между свёртками
    nn.Conv2d(in_channels=2048, out_channels=1024, kernel_size=(3,3)),
    nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=(3,3)),
    nn.ReLU(),
    nn.MaxPool2d((6,6)),  # слишком большое ядро
    nn.Conv2d(in_channels=6, out_channels=32, kernel_size=(20,20)),  # слишком большое ядро
    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(20,20)),
    nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(20,20)),
    nn.Softmax(),  # softmax в середине
    nn.Flatten(),
    nn.Linear(64, 256),  # увеличивающийся linear (нехорошо, но не фатально)
    nn.Softmax(),  # softmax в середине
    nn.Linear(256, 10),
    nn.Sigmoid(),
    nn.Dropout(0.5),  # dropout в конце
)

criterion = nn.CrossEntropyLoss()

cnn(torch.randn(2, 3, 64, 64))

RuntimeError: ignored

#### б) Задача регрессии. Предсказываем цены на недвижимость.

In [ ]:
model = nn.Sequential(
    nn.Linear(39, 128),
    # нет нелинейностей между Linear
    nn.Linear(128, 128),
    nn.Linear(128, 1),
)

criterion = nn.MSELoss()

####  б) Классификация картинок, например, fashion MNIST

In [7]:
model = nn.Sequential(
    nn.Conv2d(1, 512, kernel_size=3, padding=11),  # очень много каналов на выходе, огромный padding
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(512 * 19 * 19, 100),  # надо не 19
    nn.Softmax(),  # Softmax в середине
    nn.Dropout(0.1),
    nn.Linear(100, 10),
    nn.Softmax(),  # Softmax в модели вместо логитов / логарифмов на выходе
    nn.Dropout(0.1)  # Dropout в конце
)

criterion = nn.MSELoss()  # MSE для классификации

model(torch.randn(3, 1, 28, 28))

RuntimeError: ignored

#### в) И снова цветные картинки, размер картинки $100 \times 100$ пикселей

Если попробовать запустить эту сетку, вылезет ошибка.

In [ ]:
layers = []

filters_in = 3
for filters in [32, 64, 128, 256]:
    layers.append(nn.Conv2d(filters_in, filters, kernel_size=(5, 5)))
    # нет нелинейности
    filters_in = filters
    layers.append(nn.Conv2d(filters, filters, kernel_size=(1, 1)))
    layers.append(nn.MaxPool2d(kernel_size=(3, 3)))
    layers.append(nn.ReLU())
    # слишком быстро падает размер по H и W

layers.append(nn.Flatten())

layers.append(nn.Linear(100, 100))
layers.append(nn.ReLU())
layers.append(nn.Dropout(0.5))
layers.append(nn.Linear(100, 10))
layers.append(nn.Softmax())  # softmax в модели

model = nn.Sequential(*layers)

criterion = nn.CrossEntropyLoss()

# Book of grudges
* Input channels are wrong literally half the time (after pooling, after flatten).
* Too many filters for first 3x3 convolution - will lead to enormous matrix while there's just not enough relevant combinations of 3x3 images (overkill).
* Usually the further you go, the more filters you need.
* large filters (10x10 is generally a bad pactice, and you definitely need more than 10 of them
* the second of 10x10 convolution gets 8x6x6 image as input, so it's technically unable to perform such convolution.
* Softmax nonlinearity effectively makes only 1 or a few neurons from the entire layer to "fire", rendering 512-neuron layer almost useless. Softmax at the output layer is okay though
* Dropout after probability prediciton is just lame. A few random classes get probability of 0, so your probabilities no longer sum to 1 and crossentropy goes -inf.